<a href="https://colab.research.google.com/github/varungole/Week-In-Kaggle/blob/main/Week_In_Kaggle_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid

#neural net imports
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,id,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28,label
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [ ]:
print(f"Training Dataset: \n\tnumber of images: {train.shape[0]}\n\tnumber of features: {train.shape[1]}")
print(f"\tmissing data: {train.isnull().sum().sum()}")

Training Dataset: 
	number of images: 8000
	number of features: 786
	missing data: 0


In [ ]:
labels = train.label.value_counts().index # Get the class names
counts = train.label.value_counts() # Get the number of rows by class

print(f"Classes: {labels}")
print(f"Number of images per label: {counts}")

Classes: Int64Index([1, 2, 7, 9, 3, 0, 8, 4, 6, 5], dtype='int64')
Number of images per label: 1    921
2    826
7    812
9    806
3    803
0    791
8    783
4    782
6    767
5    709
Name: label, dtype: int64


NameError: name 'plt' is not defined

In [ ]:
X = train.drop('label' , axis = 1)
Y = test

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
#now we set datatype to float
X = X.astype('float32')
X.shape
Y = Y.astype('float32')
Y.shape

(2000, 785)

In [ ]:
X.shape


(8000, 785)

In [ ]:
#Normalize
X = X/ 255.0
Y = Y/ 255.0

In [ ]:
X.shape

(8000, 785)

In [ ]:
Y.shape

(2000, 785)

In [ ]:
y = train.label

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_val , y_train , y_val = train_test_split(X, y, test_size=0.20, random_state=16)

In [ ]:
X_train_t = torch.from_numpy(X_train).float()

In [ ]:
y_train_t =  torch.from_numpy(y_train.values)

In [ ]:
train_tensor = TensorDataset(X_train_t, y_train_t)


In [ ]:
X_val_t =  torch.from_numpy(X_val).float()
y_val_t =  torch.from_numpy(y_val.values)
val_tensor = TensorDataset(X_val_t, y_val_t)

X_test_t = torch.from_numpy(Y).float()

In [ ]:
batch_size = 16

In [ ]:
# Load image into data generator
train_loader = DataLoader(train_tensor, batch_size=batch_size, num_workers=2, shuffle=True)
val_loader = DataLoader(val_tensor, batch_size=batch_size, num_workers=2, shuffle=True)
test_loader = DataLoader(X_test_t, batch_size=batch_size, num_workers=2, shuffle=False)

loaders = {"train": train_loader, "valid": val_loader}

In [ ]:
X_train_t.size()

torch.Size([6400, 785])

In [ ]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(785, 512),  # Adjusted input size to match the number of features
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)  # Output layer with 10 classes
        )

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_stack(x)
        return output

In [ ]:
#Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
model = NeuralNetwork().to(device)


In [ ]:
learning_rate = 1e-3
batch_size = 64


In [ ]:
import torch.optim as optim

# Assuming you have already defined your model (e.g., model = ModifiedNeuralNetwork())

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:

def train_loop(data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)

    # set the model to training mode
    model.train()

    for batch, (X, y) in enumerate(data_loader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch% 100 == 0:
            loss, current  = loss, (batch + 1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:

def test_loop(data_loader, model, loss_fn):
    # set the model to evaluation mode
    model.eval()

    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in data_loader:
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:

epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(val_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.283067  [   16/ 6400]
loss: 0.409988  [ 1616/ 6400]
loss: 0.241140  [ 3216/ 6400]
loss: 0.316146  [ 4816/ 6400]
Test Error: 
 Accuracy: 91.4%, Avg loss: 0.302781 

Epoch 2
-------------------------------
loss: 0.441452  [   16/ 6400]
loss: 0.206573  [ 1616/ 6400]
loss: 0.256831  [ 3216/ 6400]
loss: 0.149029  [ 4816/ 6400]
Test Error: 
 Accuracy: 93.4%, Avg loss: 0.224426 

Epoch 3
-------------------------------
loss: 0.280874  [   16/ 6400]
loss: 0.213602  [ 1616/ 6400]
loss: 0.195640  [ 3216/ 6400]
loss: 0.026735  [ 4816/ 6400]
Test Error: 
 Accuracy: 93.8%, Avg loss: 0.221011 

Epoch 4
-------------------------------
loss: 0.001465  [   16/ 6400]
loss: 0.078437  [ 1616/ 6400]
loss: 0.204755  [ 3216/ 6400]
loss: 0.018744  [ 4816/ 6400]
Test Error: 
 Accuracy: 93.3%, Avg loss: 0.246159 

Epoch 5
-------------------------------
loss: 0.252514  [   16/ 6400]
loss: 0.116291  [ 1616/ 6400]
loss: 0.001858  [ 3216/ 6400]
loss: 0.000195  [ 4816

In [ ]:
# Prediction
logits = model(X_test_t)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 5, 6,  ..., 7, 4, 2])


In [ ]:

results = pd.Series(y_pred,name="label")

submission = pd.concat([pd.Series(range(1,28001),name = "id"),results],axis = 1)

submission.to_csv("submission.csv",index=False)

In [ ]:
submission = pd.read_csv("submission.csv")

# Display the first few rows of the DataFrame
print(submission.head())

   id  label
0   1    1.0
1   2    5.0
2   3    6.0
3   4    2.0
4   5    4.0


In [ ]:
submission = pd.read_csv("submission.csv")

# Select the first 2000 rows
subset_submission = submission.head(2000)

# Save the subset to a new CSV file
subset_submission.to_csv("subset_submission.csv", index=False)

In [62]:
daku = pd.read_csv("subset_submission.csv")

In [63]:
daku.head()

,id,label
0,1,1.0
1,2,5.0
2,3,6.0
3,4,2.0
4,5,4.0


In [64]:
daku.shape

(2000, 2)